# NLQ to SQL using watsonx and langchain

## 1. Importing Libraries

In [ ]:
# !pip install ibm-generative-ai

In [ ]:
import os
from dotenv import load_dotenv
import os

from genai.extensions.langchain import LangChainInterface
from genai.schemas import GenerateParams
from genai.credentials import Credentials

from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType

## 2. Setting up the API key and URL

In [ ]:
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_url = os.getenv("GENAI_API", None)
creds = Credentials(api_key, api_endpoint=api_url)

## 3. Defining the model and parameters

In [ ]:
model_id = "bigcode/starcoder"

In [ ]:
params = GenerateParams(
            decoding_method="greedy",
            max_new_tokens=1000,
            min_new_tokens=10,
            temperature=0.7,
        )

## 4. Creating the langchain interface

In [ ]:
llm = LangChainInterface(model=model_id, params=params, credentials=creds)

## 5. Loading the pdf data and storing it in vector database

In [ ]:
db = SQLDatabase.from_uri("sqlite:///data/chinook.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [ ]:
agent_executor.run("How many songs are in the database?")